In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [87]:
file_path = "train.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "amananandrai/ag-news-classification-dataset",
  file_path,
)
df.head()

/tmp/ipython-input-1118257203.py:6: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [88]:
file_path = "test.csv"

df_test = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "amananandrai/ag-news-classification-dataset",
  file_path,
)

df.head()

/tmp/ipython-input-2080871286.py:3: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_test = kagglehub.load_dataset(


First 5 records:    Class Index                                              Title  \
0            3  Wall St. Bears Claw Back Into the Black (Reuters)   
1            3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2            3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3            3  Iraq Halts Oil Exports from Main Southern Pipe...   
4            3  Oil prices soar to all-time record, posing new...   

                                         Description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...  


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [90]:
df["Class Index"].unique()

array([3, 4, 2, 1])

In [91]:
df['Class Index'] = df['Class Index'] -1
df_test['Class Index'] = df_test['Class Index'] -1

1-World, 2-Sports, 3-Business, 4-Sci/Tech

## Encoding Using Decoder Only model

In [92]:
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [94]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [95]:
encoded_data = model.encode(df["Description"], device=device)

In [96]:
encoded_data.shape

(120000, 768)

In [97]:
encoded_test_data = model.encode(df_test["Description"])

## Logistic Regression Head

In [99]:
l = LogisticRegression()

In [100]:
l.fit(encoded_data, df['Class Index'])

LogisticRegression()

In [101]:
y = l.predict(encoded_test_data)

In [102]:
accuracy_score(df_test['Class Index'], y)

0.9028947368421053

## Neural Network

In [105]:
class myModel(nn.Module):
  def __init__(self, in_fet, out_fet, p=.2):
    super(myModel, self).__init__()

    self.lays = nn.Sequential(
        nn.Linear(in_fet, 500),
        nn.ReLU(),
        nn.Dropout(p),
        nn.Linear(500, 400),
        nn.ReLU(),
        nn.Linear(400, 250),
        nn.ReLU(),
        nn.Dropout(p),
        nn.Linear(250, out_fet),
    )

  def forward(self, x):
    return self.lays(x)

In [106]:
class MyDataset(Dataset):
  def __init__(self, x, y):
    super().__init__()
    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.long)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [107]:
data_train = MyDataset(encoded_data, df['Class Index'])
data_test = MyDataset(encoded_test_data,  df_test['Class Index'])

In [108]:
batched_dataset = DataLoader(data_train, batch_size = 64, shuffle = True)
dataset_test = DataLoader(data_test, batch_size = 64, shuffle = True)

In [109]:
def train(epochs, model, optimizer, loss_fn, dataset_train, dataset_test):
  for epoch in range(epochs):
    train_loss = 0.0
    test_loss = 0.0
    test_i = 0
    it = 0
    for batch in dataset_train:
      x, y = batch
      x, y = x.to(device).float(), y.to(device)
      model.train()
      y_pred = model(x)
      loss = loss_fn(y_pred, y)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      train_loss += loss.item()
      it += 1

    model.eval()
    with torch.no_grad():
      for batch in dataset_test:
        x, y = batch
        x, y = x.to(device).float(), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        test_loss += loss.item()
        test_i += 1

    print(f"Epoch: {epoch}, Train Loss: {train_loss/it}, Test Loss: {test_loss/test_i}")

In [110]:
model_nn = myModel(in_fet=encoded_data.shape[1], out_fet=4).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_nn.parameters(), lr=0.001)
train(epochs=10, model=model_nn, optimizer=optimizer, loss_fn=loss_fn, dataset_train=batched_dataset, dataset_test=dataset_test)

Epoch: 0, Train Loss: 0.30748502013285955, Test Loss: 0.2806809279097228
Epoch: 1, Train Loss: 0.24823469032049178, Test Loss: 0.26087091014790936
Epoch: 2, Train Loss: 0.2168947718024254, Test Loss: 0.26093320786452096
Epoch: 3, Train Loss: 0.1888630692044894, Test Loss: 0.26062360963150233
Epoch: 4, Train Loss: 0.16590074601769447, Test Loss: 0.2708305669244097
Epoch: 5, Train Loss: 0.1456205676784118, Test Loss: 0.2685318649629084
Epoch: 6, Train Loss: 0.12980448609987894, Test Loss: 0.28268748452933895
Epoch: 7, Train Loss: 0.11742951412200928, Test Loss: 0.3193911548484774
Epoch: 8, Train Loss: 0.1061581028064092, Test Loss: 0.3188570684620312
Epoch: 9, Train Loss: 0.098263993811359, Test Loss: 0.34198082451309475


In [111]:
y_pred = model_nn(torch.tensor(encoded_test_data).to(device))

In [112]:
accuracy_score(df_test['Class Index'], torch.argmax(y_pred.cpu(), dim=1).numpy())

0.9142105263157895